In [ ]:
# [Cupoy 機器學習百日馬拉松-期末考](https://www.kaggle.com/c/ml100marathon-final-exam/)
* Version
  * V1: 使用 MobileNet 做 transfer learning
  * V2: 使用 EarlyStopping/ReduceLROnPlateau，在 training 時依 val_loss 的情況來調整 learning rate.      
  * V3: 調整最後的 FC layer，由二層調整為三層 (神經元個數也調大了)
  * V4: 調整ImageDataGenerator 的參數，做了 rotate 與平移的設定
  * V5: 改用 VGG16 做 transfer learning
  * V6: 改回原 MobileNet 做 transfer learning，但將 dataset 只分成 train/valid，用更多的 train set 來做訓練。
    * 調整 ImageDataGenerator 參數
    * 調整 epoch 30 => 50
    
* Score  

| Version | Private Score |Public Score |  
|------|------|------|
|   V1  | 0.82600 | 0.83900 |
|   V2  | 0.91600 | 0.91900 |
|   V3  | 0.92200 | 0.92100 |
|   V4  | 0.92500 | 0.94500 |
|   V5  | 0.91700 | 0.90100 |
|   V6  | 0.93100 | 0.93700 |

In [ ]:
## Version: V6

In [1]:
import os, shutil

In [2]:
img_base = '/home/ryanyao/docker_mount/data/ml100marathon-final-exam'
train_path = os.path.join(img_base, 'train')
test_path = os.path.join(img_base, 'test')
sub_train_path = os.path.join(img_base, 'sub_train')
sub_valid_path = os.path.join(img_base, 'sub_valid')
sub_test_path = os.path.join(img_base, 'sub_test')

In [3]:
class_set = os.listdir(train_path)
print(class_set)

['dandelion', 'tulip', 'rose', 'sunflower', 'daisy']


In [4]:
# Sanity checks
for i in class_set:
    print('train/%s/ images:' %(i), len(os.listdir(os.path.join(train_path, i))))

train/dandelion/ images: 687
train/tulip/ images: 633
train/rose/ images: 515
train/sunflower/ images: 488
train/daisy/ images: 500


In [5]:
# copy 50張圖片到 sub_valid, 其餘的 copy 到 sub_train
def generate_dir():
    for i_step in (sub_train_path, sub_valid_path):
        os.mkdir(i_step)
        
    for i_step in (sub_train_path, sub_valid_path):
        for i_class in class_set:
            os.mkdir(os.path.join(i_step, i_class))
            fnames = os.listdir(os.path.join(train_path, i_class))
            
            # sub_train dir
            if (i_step == sub_train_path):
                sub_fnames = fnames[:-100]
            elif (i_step == sub_valid_path):
                sub_fnames = fnames[-100:-50]
            else:
                sub_fnames = fnames[-50:]
            for fname in sub_fnames:
                src = os.path.join(train_path, i_class, fname)
                det = os.path.join(i_step, i_class, fname)
                # print(src)
                # print(det)
                shutil.copyfile(src, det)

generate_dir()

In [6]:
import numpy as np
import keras
from keras.layers.core import Dense, Flatten
from keras.layers import GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.applications.mobilenet import MobileNet, preprocess_input, decode_predictions

Using TensorFlow backend.


In [7]:
# Workaround Issue: "Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR"
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

In [8]:
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3)

In [10]:
batch_size = 32
sub_train_batches = datagen.flow_from_directory(sub_train_path, target_size=(224,224), batch_size=batch_size)
sub_valid_batches = datagen.flow_from_directory(sub_valid_path, target_size=(224,224), batch_size=batch_size)

Found 2323 images belonging to 5 classes.
Found 250 images belonging to 5 classes.


In [11]:
# 檢視 image input size
print(sub_train_batches.image_shape)

(224, 224, 3)


### 下載 Keras NobileNet model 進行 transfer learning

In [13]:
#imports the mobilenet model and discards the last 1000 neuron layer.
base_model = MobileNet(include_top=False, weights='imagenet') 

x = base_model.output
# we add dense layers so that the model can learn more complex functions and classify for better results.
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x) 
x = Dense(1024, activation='relu')(x)
preds = Dense(5, activation='softmax')(x) # final layer with softmax activation

In [14]:
model = Model(inputs=base_model.input, outputs=preds)

In [15]:
# # Check the model architecture
# for i,layer in enumerate(model.layers):
#     print(i, layer.name, layer.trainable)

In [16]:
# 輸出整個網路結構
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128       
__________

### 使用 EarlyStopping/ReduceLROnPlateau，在 training 時依 val_loss 的情況來調整 learning rate.

In [17]:
from keras.callbacks import EarlyStopping
earlystop = EarlyStopping(monitor="val_loss", patience=10, verbose=1)

from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(factor=0.3,min_lr=1e-12, monitor='val_loss', patience=5, verbose=1)

In [18]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_sub_train = sub_train_batches.n // sub_train_batches.batch_size
step_size_sub_valid = sub_valid_batches.n // sub_valid_batches.batch_size
history = model.fit_generator(generator=sub_train_batches, 
                    steps_per_epoch=step_size_sub_train, 
                    validation_data=sub_valid_batches,
                    validation_steps=step_size_sub_valid,
                    epochs=50,
                    callbacks=[earlystop,reduce_lr],
                    verbose=2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
 - 52s - loss: 0.7270 - acc: 0.7582 - val_loss: 0.8452 - val_acc: 0.7812
Epoch 2/50
 - 26s - loss: 0.4965 - acc: 0.8335 - val_loss: 0.8477 - val_acc: 0.7844
Epoch 3/50
 - 24s - loss: 0.3737 - acc: 0.8670 - val_loss: 1.3465 - val_acc: 0.6927
Epoch 4/50
 - 22s - loss: 0.4427 - acc: 0.8622 - val_loss: 0.9361 - val_acc: 0.7156
Epoch 5/50
 - 22s - loss: 0.3495 - acc: 0.8854 - val_loss: 0.6023 - val_acc: 0.7890
Epoch 6/50
 - 22s - loss: 0.3312 - acc: 0.8887 - val_loss: 0.5923 - val_acc: 0.8303
Epoch 7/50
 - 22s - loss: 0.2630 - acc: 0.9149 - val_loss: 0.8699 - val_acc: 0.7936
Epoch 8/50
 - 23s - loss: 0.2680 - acc: 0.9109 - val_loss: 0.3712 - val_acc: 0.8945
Epoch 9/50
 - 22s - loss: 0.2547 - acc: 0.9207 - val_loss: 0.3601 - val_acc: 0.9107
Epoch 10/50
 - 22s - loss: 0.2648 - acc: 0.9093 - val_loss: 0.4480 - val_acc: 0.8624
Epoch 11/50
 - 22s - loss: 0.2603 - acc: 0.9145 - val_loss: 0.8199 - val_acc: 0.8119
Epoch 12/50
 - 22s - loss:

### 上傳 Kaggle

In [19]:
print(sub_train_batches.class_indices)


{'daisy': 0, 'dandelion': 1, 'rose': 2, 'sunflower': 3, 'tulip': 4}


In [20]:
from keras.preprocessing import image
import pandas as pd

In [21]:
submit = pd.DataFrame(columns=['id', 'flower_class'])
fnames = os.listdir(test_path)

for idx, fname in enumerate(fnames):
    # Loads an image into PIL format.
    img = image.load_img(os.path.join(test_path, fname), target_size=(224, 224))
    
    # Converts a PIL Image instance to a Numpy array. shape=(224, 224, 3)
    img_data = image.img_to_array(img)   
    
    # Insert a new axis that will appear at the `axis` position in the expanded array shape. shape=(1, 224, 224, 3)
    img_data = np.expand_dims(img_data, axis=0)                

    preds = model.predict(img_data)
    submit.loc[idx] = [fname.split('.')[0], preds.argmax()]

submit.to_csv('Day101_submission_v6.csv', index=False)